# BoxCounter
## 1. Goal
Generate Data to train for BoxCounter (in DOS_2_BoxCounter)

## 2. Import neccessary libraries

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
from randDistri import V_GEN

# eigenvalue solver
from petsc4py import PETSc
from slepc4py import SLEPc

# Plot
from matplotlib import pylab

# I/O 
from numpy import savetxt, loadtxt

## 3. Setting constants

In [2]:
TEST = False

if TEST:
    BOXLENGTH = 100
    DATA_SIZE = 70 # train size, test size = DATA_SIZE//10
    NEV = 20
    PATH='test_data/'
    FILE_NAME = 't'
else:
    BOXLENGTH = 1000
    DATA_SIZE = 200 # train size, test size = DATA_SIZE//10
    NEV = 100
    PATH='data/'
    FILE_NAME = 'dos7'

NAMES = ['dos', 'evs', 'W-evs', 'W', 'W-1', 'W-2', 'DW2', 'W-1DW2', 'W-2DW2', 'V-W', 'WxV-W', 'W2xV-W']
README = 'BOXLENGTH,DATA_SIZE,NEV\n'+','.join(map(str, [BOXLENGTH, DATA_SIZE, NEV]))

## 4. Obtaining training/testing data

## Getting the eigenvalues and the landscape function

In [3]:
class DOS_data_generator():
    # self, list of int, method, int/None, int/None, bool
    def __init__(self, size, nev=NEV, V_gen=None, max_it=None, tol=None, periodic=True):
        self.max_it = max_it
        self.tol=tol
        self.periodic = True
        self.nev = nev
        
        self.V_gen = V_gen
        if V_gen==None:
            self.V_gen = np.random.rand
            
        
        if type(size) == int:
            self.size = [1,size]
        else:
            self.size = size
            
        self.one = PETSc.Vec().createSeq(self.size[1]) 
        self.one[:] = np.ones(self.size[1])
        
    # self, method --> PETSc Mat
    # Makeing a periodic problem Hamiltonian -\Delta+V
    def make_Hamiltonian(self, V):
        n = self.size[1]
        A = PETSc.Mat().create()
        A.setSizes([n, n])
        A.setUp()

        rstart, rend = A.getOwnershipRange()

        # first row
        if rstart == 0:
            A[0, :2] = [2, -1]
            rstart += 1
        # last row
        if rend == n:
            A[n-1, -2:] = [-1, 2]
            rend -= 1
        # other rows
        for i in range(rstart, rend):
            A[i, i-1:i+2] = [-1, 2+V[i], -1]
        # Periodic condition
        if self.periodic:
            A[rstart,rend-1] = -1
            A[rend-1, rstart] = -1

        A.assemble()

        return A

    # self, PETSc Mat --> np array of first nev eigenvalues
    # compute the ground state eigenvalue
    # return -1 if numerical solver is divergent
    def compute_evs(self, Hamiltonian, nev):
        E = SLEPc.EPS()
        E.create()

        E.setOperators(Hamiltonian)
        E.setProblemType(SLEPc.EPS.ProblemType.HEP)
        E.setTolerances(tol=self.tol, max_it=self.max_it)
        E.setWhichEigenpairs(E.Which.SMALLEST_REAL)
        E.setDimensions(nev=nev)

        E.solve()
        
        nconv = E.getConverged()
        #nconv = 34
        if nconv < nev:
            raise ValueError("Eigevanlue solver did not convergence for {} eigenvalue(s)".format(nev-nconv))
        
        res = np.empty(nev, dtype=np.float32)
        for i in range(nev):
            res[i] = E.getEigenvalue(i).real*(1-E.computeError(i)) # gives upper bound
        
        return res
    
    
    # self, PETSc.Mat, bool --> PETSc.Vec
    # use PETSc.Vec.getArray() to convert result to np.ndarray
    def compute_landscape(self, Hamiltonian, show=False):        
        # Create solution landscape function u
        u = PETSc.Vec().createSeq(self.size[1])
        
        # Initialize ksp solver.
        ksp = PETSc.KSP().create()
        ksp.setOperators(Hamiltonian)
        
        # Solve!
        ksp.solve(self.one, u)

        # # Use this to plot the solution (should look like a sinusoid).
        if show:
            pylab.plot(u.getArray())
            pylab.show()
            
        return u   
    
    
    # self, method --> np.ndarray, np.ndarray, np.ndarray
    # return shape: 
    # evs: data_size x nev
    # u: data_size x boxlength
    # V: data_size x boxlength
    # nev: int
    def data_gen(self, V_gen=None, nev=None):
        if V_gen == None:
            V_gen = self.V_gen
        if nev == None:
            nev = self.nev
        
        self.V = V_gen(*self.size)
        #print(self.V)
        self.W = np.empty(self.size, dtype=np.float32)
        self.evs = np.empty([self.size[0], nev], dtype=np.float32)
        
        
        
        for i in range(self.size[0]):
            Hamiltonian = self.make_Hamiltonian(self.V[i])
            self.W[i] = 1/self.compute_landscape(Hamiltonian).getArray()
            self.evs[i] = np.maximum(self.compute_evs(Hamiltonian, nev), np.min(self.W[i]))
            
    
            
        return self.evs, self.W, self.V.astype(np.float32), nev

### Example Data Generation

In [4]:
if TEST:
    DOS = DOS_data_generator([2,100], nev=10, V_gen=V_GEN)
    evs0, W0, V0, nev0 = DOS.data_gen()
    print(evs0.shape, W0.shape, V0.shape)
    print("Setting: discrete 1d integer lattice of size", BOXLENGTH)
    print("List of fist {} eigenvalues:\n".format(10), evs0)
    print("Example of the landscape potential W (orange) and V (blue):")

    pylab.plot(V0[0])
    pylab.plot(W0[0])
    pylab.show()

### Generates data

In [5]:
# int, int, method --> tf.Tensor, tf.Tensor, tf.Tensor, tf.Tensor
# generates:
# dos: (data_size x nev) x 1
# evs: (data_size x nev) x 1
# 1/u, V: (data_size x nev) x boxlength x 1
def _gen_data(data_size=DATA_SIZE, boxlength = BOXLENGTH, nev=NEV, V_gen=None):
    # error checking
    if data_size < 1:
        data_size = 1
        
    data_generator = DOS_data_generator([data_size, boxlength], nev=nev, V_gen=V_gen)
    evs, W, V, nev = data_generator.data_gen()
    
    zeros = np.zeros([data_size, nev, boxlength], dtype=np.float32)
    W = W[:, np.newaxis,:] + zeros
    W = np.concatenate(W, axis=0)
    V = V[:, np.newaxis,:] + zeros
    V = np.concatenate(V, axis=0)
    
    # (make data_size x nev) length np.array
    evs = np.concatenate(evs)
    evs = evs[..., np.newaxis]
    
    # make DOS target
    dos = np.array(range(1,nev+1), dtype=np.float32)
    dos = np.concatenate(dos+np.zeros([data_size, nev], dtype=np.float32), axis=0)
    
    #DW
    WT = W.T
    DW = WT[1:]-WT[:-1]
    DW = np.append(DW, [-W.T[-1]], 0)
    DW = DW.T
    
    return dos, evs, W-evs, W, W**(-1), W**(-2), DW**2, DW**2/W, DW**2/W**2, V-W, (V-W)/W, (V-W)/W**2 
    #, np.concatenate([W,V], axis=2)

# output is a pair, each of the format: ev, u, V, uV   
def gen_data(data_size=DATA_SIZE, nev=NEV, boxlength = BOXLENGTH, V_gen=None):
    train = _gen_data(data_size=data_size, nev=nev, boxlength=boxlength, V_gen=V_gen)
    test = _gen_data(data_size=data_size//10, nev=nev, boxlength=boxlength, V_gen=V_gen)
    
    return train, test

In [6]:
#train_ds, test_ds = gen_data(data_size=3, nev=2, boxlength=13) #test_ds 
#test_ds

### Saving/Loading

In [7]:
def save_data(train, test, names=NAMES, path=PATH, file_name=FILE_NAME, readme=README):
    with open(path+file_name+'_params.txt', 'w') as f:
        f.write(readme)
    
    for i in range(len(names)):
        savetxt(path + file_name + '_train_' + names[i] + '.cvs', train[i], delimiter=',')
        savetxt(path + file_name + '_test_' + names[i] + '.cvs', test[i], delimiter=',')
    
def load(names=NAMES, path=PATH, file_name=FILE_NAME):
    train, test= [], []
    for i in range(len(names)):
        train.append(loadtxt(path + file_name + '_train_' + names[i] + '.cvs', delimiter=','))
        test.append(loadtxt(path + file_name + '_test_' + names[i] + '.cvs', delimiter=','))
    
    return train, test

def gen_data_save(names=NAMES, data_size=DATA_SIZE, nev=NEV, boxlength = BOXLENGTH, V_gen=None, \
                path=PATH, file_name=FILE_NAME, readme=README):
    train, test = gen_data(data_size=data_size, nev=nev, boxlength = boxlength, V_gen=None)
    save_data(train, test, names=names, path=path, file_name=file_name, readme=readme)
    

In [8]:
gen_data_save()